<a href="https://colab.research.google.com/github/ni-metre/Colab_Triplet_Loss_Backup/blob/master/RealdataPickle_OldNewClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Set up gspread
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())



In [2]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

% xmode verbose

Mounted at /content/drive
Exception reporting mode: Verbose


In [ ]:
#To remoount drive
drive.mount("/content/drive", force_remount=True)

In [3]:
# Load the Pandas libraries with alias 'pd' 
import sys
import os
import os.path
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import pickle

from tensorflow.keras.preprocessing.sequence import pad_sequences

import datetime as datetime
import math
import random

In [4]:
sensor_of_interest = 9
base_name = 'Fakes_0' + str(sensor_of_interest)

rootdir = '/content/drive/My Drive/Scope_Fakes/'
#path to fake data
path = rootdir + base_name

pickle_path = '/content/drive/My Drive/Pickles'

#path to model checkpoints
checkpoint_path = '/content/drive/My Drive/Checkpoints'

#path to real data
real_path = '/content/drive/My Drive/Test_outputs/General'

In [5]:

#Run this to define import function, which gets called in each plotting function (in case Test_Log is updated)
def do_import():
  wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xHVjmlRynUE6YYZFh8rvOuFd7Mb-eWHIIwvsTvndFkY/edit?usp=sharing')
  sheet = wb.worksheet('Test_Log')
  test_data = sheet.get_all_values()
  test_db_df = pd.DataFrame(test_data)

  test_db_df.columns = test_db_df.iloc[0]
  test_db_df= test_db_df.iloc[1:]

  
  test_db_df['Sensor']= test_db_df['Sensor'].replace("?", 0)
  for column in test_db_df[['Acetone', 'File', 'CO2', 'H2O', 'Sensor', 'Test_that_day', 'Time_Btwn_s', 'Instr_Temp', 'Baseline', 'Total_CO2', 'New_Baseline', 'Sample_rate']]:
      test_db_df[column] = pd.to_numeric(test_db_df[column])
  test_db_df.set_index('File', inplace = True)
  return(test_db_df)

In [6]:
#Check that Test_Log looks okay
test_db_df = do_import()

test_db_df.fillna(0, inplace= True)
test_db_df.head()

,Raw_data,Test_name,H2O,CO2,Total_CO2,Acetone,Sample_rate,Gain,Gain_Adj,Sensor,Instr_Temp,Date,Time,Time_Btwn_s,Test_that_day,FLAG,Notes,Data_pts,Baseline,Amb_Temp,Other,Instr,Rel_Hum,EtOH_temp,Instr_T2,Instr_T3,New_Baseline,Baseline_Dip,Peak_neg,Peak_pos,Peak_neg_t,Peak_pos_t,Cross_t,Cross_t_ix,Peak_neg_t_ix,Peak_pos_t_ix,Area_neg_raw,Area_pos_raw,Area_neg,Area_pos_X,...,Area_to_Cal_int,Area_Pos_to_Cal_int,Baseline_slope,slope_calc,flag_slope,a_,b_,c_,del_a,del_b,del_c,cpn_time,cpn_mV,cpn_manual_mV,cpn_manual_time,solve,solve2,del_pnt,del_ppt,del_xtime,del_cpnt,del_pn,del_pp,slope_pos_peak,slope_pos_peak_t,slope_pos_peak_t_ix,slope_neg_peak,slope_neg_peak_t,slope_neg_peak_t_ix,slope_cross_t,slope_cross_t_ix,dx2_pos_peak,dx2_pos_peak_t,dx2_pos_peak_t_ix,dx2_neg_peak,dx2_neg_peak_t,dx2_neg_peak_t_ix,dx2_cross_t,dx2_cross_t_ix,dx2_cross_t_ix
File,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3000,wav3000,Nitrogen blank,0,0,0.0,0.0,250,,,7,0.0,2020-02-11,1:53:44 PM,50024.0,1,,,8076,26.770204,0,0.00,38107,0.00,0,,,26.0,-0.84,-2.495264403,34.0613,18.4200,1.32,0.00,0,1842.00000,132,0.00,-1060.00,0.00,0.00,...,0,0,0.13728,-0.73976,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.6115311406,0.3,30,-0.1127246342,2.58,258,6.98,698,0.03976674596,0.21,21,-0.01064621545,0.45,45,1.88,188,188
3001,wav3000,Nitrogen blank,0,0,0.0,0.0,250,,,6,0.0,2020-02-11,1:54:02 PM,50042.0,1,,,6253,25.223295,0,0.00,38263,0.00,0,,,25.0,0.0446,-3.68,4.53E+01,10.3000,1.14E+00,0.00,0.00E+00,1030.00000,1.14E+02,0.00,1737.00,0.00,0.00,...,0,0,0.07312,-0.6294,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9091349749,0.29,29,-0.145461596,2.57,257,8.25,825,0.06433878284,0.21,21,-0.01647321493,0.49,49,1.8,180,180
3002,wav3001,Nitrogen blank,0,0,0.0,0.0,250,,,7,0.0,2020-02-11,1:56:27 PM,163.0,2,,,5380,26.311880,0,0.00,38107,0.00,0,,,26.0,0.0605,-2.476566964,38.7806,18.0500,1.27,0.00,0,1805.00000,127,0.00,5373.00,0.00,0.00,...,0,0,0.01004,-0.67635,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.721437659,0.29,29,-0.1183608659,2.76,276,7.92,792,0.05009983743,0.22,22,-0.01189871139,0.48,48,1.76,176,176
3003,wav3001,Nitrogen blank,0,0,0.0,0.0,250,,,6,0.0,2020-02-11,1:56:36 PM,154.0,2,,,4455,25.250096,0,0.00,38263,0.00,0,,,25.0,-0.0134,-3.71,4.56E+01,11.8000,1.15E+00,0.00,0.00E+00,1180.00000,1.15E+02,0.00,5217.00,0.00,0.00,...,0,0,0,-0.3147,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9063376365,0.3,30,-0.1398669192,2.44,244,8.37,837,0.06278470596,0.22,22,-0.01491913805,0.52,52,1.72,172,172
3004,wav3002,Nitrogen blank,0,0,0.0,0.0,250,,,7,0.0,2020-02-11,2:04:19 PM,472.0,3,,,4184,26.724979,0,0.00,38107,0.00,0,,,27.0,0.1782,-2.078048257,32.9568,18.1300,1.29,0.00,0,1813.00000,129,0.00,6489.00,0.00,0.00,...,0,0,0.0067,-0.67635,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.6058949089,0.3,30,-0.1042702866,2.47,247,7.64,764,0.04383735775,0.22,22,-0.009393719518,0.48,48,1.75,175,175


In [7]:
#Returns a list of tests that meet ALL of the criteria defined in the arguments
# Acetone can be a specific value, 'all', '>(value)' or '<(value)'
# CO2 can be a specific value, 'all' or '+' for CO@ values that are > 0
# H2O must be a specific value, almost always 10 for tests of interest
# Sensor can be a specific sensor number or 'all'
# Range1/2 is the lower and upper limit for the range of tests of interest from range1 to rang2 - 1

def get_test_list (df, Acetone, CO2, H2O, Sensor, Range1, Range2):

    if Sensor == 'all':
        filtered = df
    else:
        filtered = df[(df['Sensor'] == Sensor)]
    filtered = filtered.query('FLAG ==""')
    if Acetone != 'all':
        if str(Acetone)[:1] == '<':
            filtered = filtered[filtered['Acetone'].astype(float) < float(Acetone[1:])]
        else:
            if str(Acetone)[:1] == '>':
                filtered = filtered[filtered['Acetone'].astype(float) > float(Acetone[1:])]
            else:
                filtered = filtered[(filtered['Acetone'] == Acetone)]
    if CO2 != 'all':
        if CO2 == '+':
            filtered = filtered[(filtered['CO2'] != '0')]
        else:
            filtered = filtered[(filtered['CO2'] == CO2)]
    filtered = filtered[(filtered['H2O'] == H2O)]
    filtered['Tests'] = filtered.index  
    list_of_tests = filtered['Tests'].tolist()
    return([y for y in list_of_tests if ((int(y)>=Range1)&(int(y)<Range2))])
#Returns a list of tests that meet ALL of the criteria defined in the arguments
# SAME AS get_test_list above for the following
# Acetone can be a specific value, 'all', '>(value)' or '<(value)'
# CO2 can be a specific value, 'all' or '+' for CO@ values that are > 0
# H2O must be a specific value, almost always 10 for tests of interest
# Sensor can be a specific sensor number or 'all'
# Range1/2 is the lower and upper limit for the range of tests of interest from range1 to rang2 - 1
#BUT
# temp is a temperature value in degrees C
# and delta is range in degrees C above and below temp that will be included in the returned list

def get_test_list_Temp (df, Acetone, CO2, H2O, temp, delta, Sensor, Range1, Range2):

    if Sensor == 'all':
        filtered = df
    else:
        filtered = df[(df['Sensor'] == Sensor)]
    filtered = filtered.query('FLAG ==""')
    if Acetone != 'all':
        if str(Acetone)[:1] == '<':
            filtered = filtered[filtered['Acetone'].astype(float) < float(Acetone[1:])]
        else:
            if str(Acetone)[:1] == '>':
                filtered = filtered[filtered['Acetone'].astype(float) > float(Acetone[1:])]
            else:
                filtered = filtered[(filtered['Acetone'] == Acetone)]
    if CO2 != 'all':
        if CO2 == '+':
            filtered = filtered[(filtered['CO2'] != '0')]
        else:
            filtered = filtered[(filtered['CO2'] == CO2)]
    filtered = filtered[(filtered['H2O'] == H2O)]
    filtered['Tests'] = filtered.index
    filtered['Instr_Temp'] = pd.to_numeric(filtered['Instr_Temp'])
    filtered = filtered[filtered['Instr_Temp']>= temp - delta]
    filtered = filtered[filtered['Instr_Temp']<= temp + delta]
    list_of_tests = filtered['Tests'].tolist()
    return([y for y in list_of_tests if ((int(y)>=Range1)&(int(y)<Range2))])

In [12]:
sos_list = [9 , 10 , 14, 15, 23, 24]
maxtco2_list = [250, 250, 137, 137, 238, 238]

In [13]:
for i in range(0, len(sos_list)):
  sos = sos_list[i]
  print(sos)
  maxtco2 = maxtco2_list[i]
  real_test_set = get_test_list (test_db_df, 'all', '+', 10, sos, 3300, 20000)
  print(len(real_test_set))
  scale = 1
  cut_off_time = 30
  scale_factor = 1

  real_test_list = []
  real_Ac_list = []
  real_co2_list = []
  real_temp_list = []
  real_bl_list = []
  real_ttd_list = []
  real_tbtwn_list = []
  real_sensor_list = []
  labelList = []
  real_tco2_list = []

  for j in real_test_set:
          ac = float(test_db_df.loc[j, 'Acetone'])
          co2 = int(test_db_df.loc[j, 'CO2'])/(2000*scale_factor)
          temp = float(test_db_df.loc[j, 'Instr_Temp'])
          bs = float(test_db_df.loc[j, 'Baseline'])
          timebtwn = float(test_db_df.loc[j, 'Time_Btwn_s'])
          totalco2 = float(test_db_df.loc[j, 'Total_CO2'])
          if totalco2 >= 100:
            labelval = 1.0
          else:
            labelval = 0.0
          testday = float(test_db_df.loc[j, 'Test_that_day'])
          sample_rate = int(test_db_df.loc[j, 'Sample_rate'])
          num_of_samples = int(cut_off_time*sample_rate/scale)

          full_data = pd.read_csv(real_path+ '/test_output' + str(int(j)) + '.csv',header = 0)
          sig_data = full_data[['Second','mV_rolling_mean']].truncate(after = cut_off_time*sample_rate-1)
          mV_data = sig_data['mV_rolling_mean'].to_numpy()
          mV_data[np.isnan(mV_data)]=0

          #if sample_rate == 500:
          if sample_rate == 500:
              mV_data = mV_data.reshape(-1, 2).mean(axis=1)

          real_test_list.append(mV_data)
          real_Ac_list.append(ac)
          real_co2_list.append(co2)
          real_temp_list.append(temp)
          labelList.append(labelval)
          real_bl_list.append(bs)
          real_ttd_list.append(testday)
          real_tbtwn_list.append(timebtwn)
          real_tco2_list.append(totalco2)
          real_sensor_list.append(sos)
  X_real = np.squeeze(np.array(real_test_list))[...,np.newaxis]
  X_real_n = pad_sequences(X_real, maxlen = 7500)

  real_data_dict = {'testnum': np.array(real_test_set), 'csvs':  np.array(X_real_n), 'label':  np.array(labelList), 'CO2':  np.array(real_co2_list), 'Ac':  np.array(real_Ac_list), 
                    'temp':  np.array(real_temp_list), 'baseline':  np.array(real_bl_list), 'ttd':  np.array(real_ttd_list), 'timebtwn':  np.array(real_tbtwn_list), 'sensor':  np.array(real_sensor_list), 
                    'tco2': np.array(real_tco2_list), 'norm_tco2': np.array(real_tco2_list)/maxtco2} 
  with open(pickle_path +'/realdatadict_oldnewclassifier_' + str(sos) + '.pickle', 'wb') as handle:
    pickle.dump(real_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

9
404
10
398
14
250
15
266
23
472
24
497


In [65]:
for i in range[0, len(sos_list)]:
  
real_test_set = get_test_list (test_db_df, 'all', '+', 10, sos, 3300, 20000)
print(len(real_test_set))

404


In [66]:

scale = 1
cut_off_time = 30
scale_factor = 1

real_test_list = []
real_Ac_list = []
real_co2_list = []
real_temp_list = []
real_bl_list = []
real_ttd_list = []
real_tbtwn_list = []
real_sensor_list = []
labelList = []
real_tco2_list = []

for j in real_test_set:
        ac = float(test_db_df.loc[j, 'Acetone'])
        co2 = int(test_db_df.loc[j, 'CO2'])/(2000*scale_factor)
        temp = float(test_db_df.loc[j, 'Instr_Temp'])
        bs = float(test_db_df.loc[j, 'Baseline'])
        timebtwn = float(test_db_df.loc[j, 'Time_Btwn_s'])
        totalco2 = float(test_db_df.loc[j, 'Total_CO2'])
        if totalco2 >= 100:
          labelval = 1.0
        else:
          labelval = 0.0
        testday = float(test_db_df.loc[j, 'Test_that_day'])
        sample_rate = int(test_db_df.loc[j, 'Sample_rate'])
        num_of_samples = int(cut_off_time*sample_rate/scale)
    
        full_data = pd.read_csv(real_path+ '/test_output' + str(int(j)) + '.csv',header = 0)
        sig_data = full_data[['Second','mV_rolling_mean']].truncate(after = cut_off_time*sample_rate-1)
        mV_data = sig_data['mV_rolling_mean'].to_numpy()
        mV_data[np.isnan(mV_data)]=0
               
        #if sample_rate == 500:
        if sample_rate == 500:
            mV_data = mV_data.reshape(-1, 2).mean(axis=1)
       
        real_test_list.append(mV_data)
        real_Ac_list.append(ac)
        real_co2_list.append(co2)
        real_temp_list.append(temp)
        labelList.append(labelval)
        real_bl_list.append(bs)
        real_ttd_list.append(testday)
        real_tbtwn_list.append(timebtwn)
        real_tco2_list.append(totalco2)
        real_sensor_list.append(sos)
X_real = np.squeeze(np.array(real_test_list))[...,np.newaxis]
X_real_n = pad_sequences(X_real, maxlen = 7500)

real_data_dict = {'testnum': np.array(real_test_set), 'csvs':  np.array(X_real_n), 'label':  np.array(labelList), 'CO2':  np.array(real_co2_list), 'Ac':  np.array(real_Ac_list), 
                  'temp':  np.array(real_temp_list), 'baseline':  np.array(real_bl_list), 'ttd':  np.array(real_ttd_list), 'timebtwn':  np.array(real_tbtwn_list), 'sensor':  np.array(real_sensor_list), 
                  'tco2': np.array(real_tco2_list), 'norm_tco2': np.array(real_tco2_list)/maxtco2}


In [67]:
with open(pickle_path +'/realdatadict_oldnewclassifier_' + str(sos) + '.pickle', 'wb') as handle:
    pickle.dump(real_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)